In [2]:
import requests
import json
import time

# Base URL Layer
base_url = "https://kspservices.big.go.id/satupeta/rest/services/PUBLIK/PERENCANAAN_RUANG/MapServer/12/query"

# Daftar target daerah
targets = ["KOTA TANGERANG SELATAN", "OGAN KOMERING ULU"]

def download_rtrw(wilayah):
    print(f"--- Memulai download untuk: {wilayah} ---")
    
    # Parameter query
    params = {
        'where': f"wadmkk = '{wilayah}'",
        'outFields': '*',
        'f': 'geojson'
    }
    
    # Coba ambil data pertama kali
    r = requests.get(base_url, params=params)
    data = r.json()
    
    all_features = []
    
    # Cek apakah ada data
    if 'features' not in data or len(data['features']) == 0:
        print(f"Data tidak ditemukan untuk nama: {wilayah}. Coba variasi nama lain.")
        return

    # Cek apakah kena limit transfer
    if 'exceededTransferLimit' in data and data['exceededTransferLimit']:
        print(f"Data {wilayah} terlalu besar. Melakukan pengambilan bertahap (Pagination)...")
        
        # Ambil total count dulu
        params_count = {
            'where': f"wadmkk = '{wilayah}'",
            'returnCountOnly': 'true',
            'f': 'json'
        }
        r_count = requests.get(base_url, params=params_count)
        total_count = r_count.json()['count']
        print(f"Total polygon untuk {wilayah}: {total_count}")
        
        # Looping pengambilan data
        offset = 0
        limit = 1000
        while offset < total_count:
            params_loop = {
                'where': f"wadmkk = '{wilayah}'",
                'outFields': '*',
                'resultOffset': offset,
                'resultRecordCount': limit,
                'f': 'geojson'
            }
            r_loop = requests.get(base_url, params=params_loop)
            data_loop = r_loop.json()
            
            if 'features' in data_loop:
                all_features.extend(data_loop['features'])
                print(f"Berhasil ambil batch {offset} s/d {offset+limit}")
            else:
                break
            
            offset += limit
            
    else:
        # Jika tidak kena limit, ambil langsung
        all_features = data['features']
        print(f"Berhasil mengambil data {wilayah} sekaligus.")

    # Simpan ke file GeoJSON
    output = {
        "type": "FeatureCollection",
        "features": all_features
    }
    
    filename = f"RTRW_{wilayah.replace(' ', '_')}.geojson"
    with open(filename, "w") as f:
        json.dump(output, f)
    
    print(f"File disimpan: {filename}\n")

# Jalankan untuk kedua daerah
for target in targets:
    download_rtrw(target)
    time.sleep(1) # Jangan spam server terlalu cepat

--- Memulai download untuk: KOTA TANGERANG SELATAN ---
Berhasil mengambil data KOTA TANGERANG SELATAN sekaligus.
File disimpan: RTRW_KOTA_TANGERANG_SELATAN.geojson

--- Memulai download untuk: OGAN KOMERING ULU ---
Berhasil mengambil data OGAN KOMERING ULU sekaligus.
File disimpan: RTRW_OGAN_KOMERING_ULU.geojson



In [3]:
import requests
import json

base_url = "https://kspservices.big.go.id/satupeta/rest/services/PUBLIK/PERENCANAAN_RUANG/MapServer/12/query"

# Kita cari di Provinsi Banten
params = {
    'where': "wadmpr = 'BANTEN'", # Filter Provinsi Banten
    'outFields': 'wadmkk',       # Kita hanya butuh nama Kab/Kota
    'returnDistinctValues': 'true', # Penting: Jangan ulang nama yang sama
    'returnGeometry': 'false',    # Jangan ambil koordinat (biar cepat)
    'f': 'json'
}

r = requests.get(base_url, params=params)
data = r.json()

if 'features' in data:
    print("Daftar Nama Kab/Kota di Banten (versi database BIG):")
    print("-" * 50)
    unique_names = []
    for feature in data['features']:
        nama = feature['attributes']['wadmkk']
        unique_names.append(nama)
    
    # Urutkan agar mudah dibaca
    unique_names.sort()
    
    for nama in unique_names:
        print(nama)
else:
    print("Gagal mengambil data.")

Daftar Nama Kab/Kota di Banten (versi database BIG):
--------------------------------------------------
KOTA CILEGON
KOTA SERANG
KOTA TANGERANG
KOTA TANGERANG SELATAN
LEBAK
PANDEGLANG
SERANG
TANGERANG
